# Challenge: Implement the Nearest Neighbor Algorithm¶
The Nearest Neighbor algorithm is extremely simple. So simple, in fact, that you should be able to build it yourself from scratch using the Python you already know. Code a Nearest Neighbors algorithm that works for two dimensional data. You can use either arrays or dataframes to do this. Test it against the SKLearn package on the music dataset from above to ensure that it's correct.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from math import sqrt

from sklearn.neighbors import KNeighborsClassifier

%matplotlib inline

In [2]:
# Some data to play with
music= pd.DataFrame()

music['duration'] = [184, 134, 243, 186, 122, 197, 294, 382, 102, 264, 
                     205, 110, 307, 110, 397, 153, 190, 192, 210, 403,
                     164, 198, 204, 253, 234, 190, 182, 401, 376, 102]

music['loudness'] = [18, 34, 43, 36, 22, 9, 29, 22, 10, 24, 
                     20, 10, 17, 51, 7, 13, 19, 12, 21, 22,
                     16, 18, 4, 23, 34, 19, 14, 11, 37, 42]

# We know whether the songs in our training data are jazz or not
music['jazz'] = [ 1, 0, 0, 0, 1, 1, 0, 1, 1, 0,
                  0, 1, 1, 0, 1, 1, 0, 1, 1, 1,
                  1, 1, 1, 1, 0, 0, 1, 1, 0, 0]

In [3]:
temp = music[['loudness', 'duration', 'jazz']]
music_array = np.array(temp)

In [4]:
music_array

array([[ 18, 184,   1],
       [ 34, 134,   0],
       [ 43, 243,   0],
       [ 36, 186,   0],
       [ 22, 122,   1],
       [  9, 197,   1],
       [ 29, 294,   0],
       [ 22, 382,   1],
       [ 10, 102,   1],
       [ 24, 264,   0],
       [ 20, 205,   0],
       [ 10, 110,   1],
       [ 17, 307,   1],
       [ 51, 110,   0],
       [  7, 397,   1],
       [ 13, 153,   1],
       [ 19, 190,   0],
       [ 12, 192,   1],
       [ 21, 210,   1],
       [ 22, 403,   1],
       [ 16, 164,   1],
       [ 18, 198,   1],
       [  4, 204,   1],
       [ 23, 253,   1],
       [ 34, 234,   0],
       [ 19, 190,   0],
       [ 14, 182,   1],
       [ 11, 401,   1],
       [ 37, 376,   0],
       [ 42, 102,   0]])

In [5]:
# Calculate euclidean distance between two points
def euclidean_distance(row1, row2):
    distance = 0.0
    for i in range(len(row1) - 1): # Ignores last column which is the classification
        distance += (row1[i] - row2[i])**2 # Squared distance between two points
    return sqrt(distance)

In [6]:
# Test euclidean_distance between first point and all other points
row1 = music_array[0]

for row in music_array:
    distance = euclidean_distance(row1, row)
    print(distance)

0.0
52.49761899362675
64.07807737440318
18.110770276274835
62.12889826803627
15.811388300841896
110.54863183232979
198.04039991880444
82.38931969618392
80.22468448052632
21.095023109728988
74.43117626371358
123.00406497347964
81.02468759581859
213.283848427395
31.400636936215164
6.082762530298219
10.0
26.1725046566048
219.03652663425797
20.09975124224178
14.0
24.413111231467404
69.18092222571191
52.49761899362675
6.082762530298219
4.47213595499958
217.1128738697915
192.93781381574738
85.44003745317531


In [7]:
# Get neighbors and sort by distance (nearest neighbors)
def get_neighbors(train_data, test_row,  num_neighbors):
    distances = list()
    for train_row in train_data:
        dist = euclidean_distance(test_row, train_row)
        distances.append((train_row, dist))
    distances.sort(key=lambda tup: tup[1]) # sort list of train_row and dist tuples by dist tuple
    neighbors = list()
    for i in range(num_neighbors):
        neighbors.append(distances[i][0])
    return neighbors

In [8]:
# Test get_neighbors between first point and all other points
neighbors = get_neighbors(music_array, music_array[0], 4)
for neighbor in neighbors:
    print(neighbor)

[ 18 184   1]
[ 14 182   1]
[ 19 190   0]
[ 19 190   0]


In [23]:
# Make predictions (limited to 2D data)
def predict(train_data, test_row, num_neighbors):
    test_row.append(2)
    neighbors = get_neighbors(train_data, test_row, num_neighbors)
    output = [row[-1] for row in neighbors] # get classification for each neighbor
    prob_one = sum(output) / len(output)
    prob_zero = 1 - prob_one
    if prob_one > prob_zero:
        prediction = 1
    elif prob_one < prob_zero:
        prediction = 0
    elif prob_one == prob_zero:
        prediction = 'either'
    return [prob_zero, prob_one], prediction

In [24]:
# Test nearest neighbor algorithm
predict(music_array, [24, 190], 5)

([0.4, 0.6], 1)

Adapted from example at https://machinelearningmastery.com/tutorial-to-implement-k-nearest-neighbors-in-python-from-scratch/